## 1. PP-TinyPose Introduction
PP-TinyPose is a real-time keypoint detection model optimized by PaddleDetecion for mobile devices, which can smoothly run multi-person pose estimation tasks on mobile devices. With the excellent self-developed lightweight detection model [PicoDet](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/picodet/README.md)，

we also provide a lightweight pedestrian detection model. PP-TinyPose has the following dependency requirements:
PaddlePaddle>=2.2

If you want to deploy it on the mobile devives, you also need:

Paddle-Lite>=2.11

More deployment cases can be referred to[PP-TinyPose](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/configs/keypoint/tiny_pose/README.md)

## 2. Model effects and application scenarios
### 2.1 Key point detection task：

#### 2.1.1 dataset

The current Keypoint model supports[COCO](https://cocodataset.org/#keypoints-2017) and [MPII](http://human-pose.mpi-inf.mpg.de/#overview).Please refer to[Key point data preparation](https://github.com/PaddlePaddle/PaddleDetection/blob/release/2.5/docs/tutorials/KeyPointConfigGuide_en.md)

#### 2.1.2 Model effects：


![](https://user-images.githubusercontent.com/15810355/181733705-d0f84232-c6a2-43dd-be70-4a3a246b8fbc.gif)




## 3. How to use the model

### 3.1 model Inference：

（When not running on Jupyter Notebook, you need to set "!" or "%" removed）


In [ ]:
%cd ~/work
# clone PaddleDetection。
!git clone https://gitee.com/paddlepaddle/PaddleDetection

* install

In [ ]:
# 
%cd ~/work/PaddleDetection/

# 
!pip install pyzmq   
!pip install -r requirements.txt

#
%cd ~/work/PaddleDetection/
# 
%env PYTHONPATH=.:$PYTHONPATH
# set GPU
%env CUDA_VISIBLE_DEVICES=0

# 
!pip install pycocotools  
!pip install cython-bbox      
!pip install xmltodict  
!pip install terminaltables    
!pip intall motmetrics  
!pip install lap    
!pip install shapely      
!pip install pytest-benchmark    
!pip install pytest    


# PaddleDetection 
!python setup.py install  

* Verify whether the installation is successful. If an error is reported, just perform the previous step

In [ ]:

!python ppdet/modeling/tests/test_architectures.py

* quickly start

congratulations! You have successfully installed PaddleDetection. Next, we will quickly detect the effect of key points. You can directly download the corresponding predictive deployment model provided in the model base, obtain the predictive deployment models of pedestrian detection model and key point detection model respectively, and decompress them.

In [ ]:
# Download model
!mkdir output_inference
%cd output_inference
# Download pedestrian detection model
!wget https://bj.bcebos.com/v1/paddledet/models/keypoint/tinypose_enhance/picodet_s_320_lcnet_pedestrian.zip
!unzip picodet_s_320_lcnet_pedestrian.zip
# Download key point detection model
!wget https://bj.bcebos.com/v1/paddledet/models/keypoint/tinypose_enhance/tinypose_128x96.zip
!unzip tinypose_128x96.zip

%cd ~/work/PaddleDetection/
# Predict a image
!python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --image_file=demo/hrnet_demo.jpg --device=GPU
# predict multiple images
!python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --image_dir=demo/ --device=GPU

# predict video
!python3 deploy/python/det_keypoint_unite_infer.py --det_model_dir=output_inference/picodet_s_320_pedestrian --keypoint_model_dir=output_inference/tinypose_128x96 --video_file={your video file} --device=GPU


### 3.2 Train：
*  clone PaddleDetection refer 3.1

* Dataset preparation

    The training set of key point detection model and pedestrian detection model include `COCO` and[AI Challenger](https://arxiv.org/abs/1711.06475)，The key points of each dataset are defined as follows:
    ```
    COCO keypoint Description:
        0: "Nose",
        1: "Left Eye",
        2: "Right Eye",
        3: "Left Ear",
        4: "Right Ear",
        5: "Left Shoulder,
        6: "Right Shoulder",
        7: "Left Elbow",
        8: "Right Elbow",
        9: "Left Wrist",
        10: "Right Wrist",
        11: "Left Hip",
        12: "Right Hip",
        13: "Left Knee",
        14: "Right Knee",
        15: "Left Ankle",
        16: "Right Ankle"

    AI Challenger Description:
        0: "Right Shoulder",
        1: "Right Elbow",
        2: "Right Wrist",
        3: "Left Shoulder",
        4: "Left Elbow",
        5: "Left Wrist",
        6: "Right Hip",
        7: "Right Knee",
        8: "Right Ankle",
        9: "Left Hip",
        10: "Left Knee",
        11: "Left Ankle",
        12: "Head top",
        13: "Neck"
    ```

    Since the annatation format of these two datasets are different, we aligned their annotations to `COCO` format. You can download [Training List](https://bj.bcebos.com/v1/paddledet/data/keypoint/aic_coco_train_cocoformat.json) and put it at `dataset/`. To align these two datasets, we mainly did the following works:
    - Align the indexes of the `AI Challenger` keypoint to be consistent with `COCO` and unify the flags whether the keypoint is labeled/visible.
    - Discard the unique keypoints in `AI Challenger`. For keypoints not in this dataset but in `COCO`, set it to not labeled.
    - Rearranged `image_id` and `annotation id`.
   
    If you need to customize the dataset, you can refer to[Quick Start - Custom Dataset](https://github.com/PaddlePaddle/PaddleDetection/tree/release/2.5/configs/keypoint#%E5%BF%AB%E9%80%9F%E5%BC%80%E5%A7%8B)
    

In [ ]:
# keypoint detection model
!python3 -m paddle.distributed.launch tools/train.py -c configs/keypoint/tiny_pose/tinypose_128x96.yml

# pedestrian detection model
!python3 -m paddle.distributed.launch tools/train.py -c configs/picodet/application/pedestrian_detection/picodet_s_320_lcnet_pedestrian.yml

## 4. Solution
<div align="center">
  <img src="tinypose_pipeline.png" width='800'/>
</div>